In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# tempat file

In [23]:
# file_url = 'https://drive.google.com/drive/u/0/folders/1ML46U6VaL9h9yfQD-2PFCSkanb74eZB1'



# Import Data

In [2]:
train = pd.read_csv('/kaggle/input/dataset-datathon/train_fix.csv')
test = pd.read_csv('/kaggle/input/dataset-datathon/test_fix (1).csv')
test.drop(columns=('Unnamed: 0'),inplace=True)
train.drop(columns=('Unnamed: 0'),inplace=True)

In [3]:
# train = train.drop(columns=['waktu_setempat'])
ntrain = train.shape[0]
ntest = test.shape[0]
y_train_final = train.rerata_kecepatan.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['rerata_kecepatan'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (526137, 41)


# Data Preprocessing & Feature Engineering

In [4]:
all_data['waktu_setempat'] = pd.to_datetime(all_data['waktu_setempat'])
all_data['hari_minggu'] = all_data['waktu_setempat'].dt.day_of_week
all_data['jam'] = all_data['waktu_setempat'].dt.hour
# Definisikan fungsi untuk mengelompokkan waktu
def categorize_time_of_day(hour):
    if 7 <= hour < 10:
        return 1
    elif 10 <= hour < 16:
        return 2
    elif 16 <= hour < 19:
        return 3
    else:
        return 4

# Tambahkan kolom kategori waktu
all_data['waktu'] = all_data['waktu_setempat'].dt.hour.apply(categorize_time_of_day)

# Definisikan fungsi untuk mengelompokkan akhir pekan atau bukan
def categorize_weekend(day):
    if day.day_of_week < 5:  # 0-4 adalah Senin-Jumat
        return 1
    else:
        return 2

# Tambahkan kolom kategori akhir pekan
all_data['weekend'] = all_data['waktu_setempat'].apply(categorize_weekend)
all_data['Latitude_awal'] = all_data['Latitude_awal'].astype('float')
all_data['Longitude_awal'] = all_data['Longitude_awal'].astype('float')
all_data['Latitude_akhir'] = all_data['Latitude_akhir'].astype('float')
all_data['Longitude_akhir'] = all_data['Longitude_akhir'].astype('float')
all_data.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,Latitude_awal,Longitude_awal,Latitude_akhir,Longitude_akhir,highway,lit,maxspeed,name,not:name,ref,lanes,lanes:backward,lanes:forward,operator,source:name,source:ref,bicycle,foot,lane_markings,sidewalk,cycleway:left,cycleway:right,cycleway,surface,ref:GB:usrn,sidewalk:both,source:maxspeed,wikidata,maxspeed:type,pavement:minimum_width,source,class:bicycle:commute,horse,oneway,busway,turn:lanes:forward,id,hari_minggu,jam,waktu,weekend
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,51.434928,-0.161176,51.434975,-0.161090,primary,yes,30 mph,High Road,Dalrymple Close,A1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,4,2
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,51.623990,-0.176398,51.623811,-0.176424,trunk,yes,30 mph,Roehampton Lane,NaN,A306,3.0,2.0,1.0,Transport for London,photograph,photograph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,4,2
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,51.452530,-0.152437,51.452749,-0.151896,trunk,yes,30 mph,Upper Richmond Road,NaN,A205,NaN,NaN,NaN,Transport for London,local knowledge,photograph,yes,yes,no,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,4,2
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,51.465851,-0.154188,51.465654,-0.155334,trunk,yes,30 mph,Upper Richmond Road West,NaN,A205,2.0,NaN,NaN,Transport for London,NaN,NaN,yes,yes,NaN,both,lane,lane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,4,2
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,51.530289,-0.228343,51.530197,-0.227949,primary,yes,20 mph,Acre Lane,NaN,A2217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,NaN,NaN,lane,asphalt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,4,2


# Handling Missing Value

In [5]:
data_stat = pd.DataFrame()
data_stat.index = all_data.columns
data_stat["unique_value"] = all_data.nunique()
data_stat["missing_rate"] = all_data.isna().mean()
data_stat["dtype"] = all_data.dtypes
data_stat

,unique_value,missing_rate,dtype
waktu_setempat,695,0.000000,"datetime64[ns, UTC]"
id_jalan,20,0.000000,int64
id_titik_mulai,488,0.000000,int64
id_titik_akhir,488,0.000000,int64
Latitude_awal,481,0.014753,float64
Longitude_awal,479,0.014753,float64
Latitude_akhir,481,0.014416,float64
Longitude_akhir,479,0.014416,float64
highway,3,0.000000,object
lit,1,0.000000,object


In [6]:
# List kolom yang ingin diimputasi
cols_to_impute = ['Latitude_awal', 'Longitude_awal','Latitude_akhir', 'Longitude_akhir']

# Mengisi nilai null dengan rata-rata pada kolom yang diinginkan
for col in cols_to_impute:
    col_mean = all_data[col].median()
    all_data[col].fillna(col_mean, inplace=True)

In [7]:
all_data_clean = all_data
data_stat = pd.DataFrame()
data_stat.index = all_data_clean.columns
data_stat["unique_value"] = all_data_clean.nunique()
data_stat["missing_rate"] = all_data_clean.isna().mean()
data_stat["dtype"] = all_data_clean.dtypes
# Kolom yang datanya hilang > 50%
miss_col = data_stat[data_stat["missing_rate"] > 0.5].index.to_list()
print("Kolom yang datanya hilang > 50%:")
print(miss_col)
print()

# Kolom yang terlalu unik
vari_col = data_stat[data_stat["unique_value"] == all_data_clean.shape[0]].index.to_list()
print("Kolom yang terlalu unik:")
print(vari_col)
print()

# Kolom dengan kategori yang banyak
cat_col_stat = data_stat[data_stat["dtype"] == "object"]
vari_cat_col = cat_col_stat[cat_col_stat["unique_value"] > 1000].index.to_list()
print("Kolom dengan kategori yang banyak:")
print(vari_cat_col)
print()


removed_features = miss_col + vari_col + vari_cat_col 

Kolom yang datanya hilang > 50%:
['not:name', 'lanes', 'lanes:backward', 'lanes:forward', 'operator', 'source:name', 'source:ref', 'bicycle', 'foot', 'lane_markings', 'cycleway:left', 'cycleway:right', 'cycleway', 'ref:GB:usrn', 'sidewalk:both', 'source:maxspeed', 'wikidata', 'maxspeed:type', 'pavement:minimum_width', 'source', 'class:bicycle:commute', 'horse', 'oneway', 'busway', 'turn:lanes:forward', 'id']

Kolom yang terlalu unik:
[]

Kolom dengan kategori yang banyak:
[]



In [8]:
# Hilangkan fitur yang tidak terpakai
all_data_clean = all_data_clean.loc[:, ~all_data_clean.columns.isin(removed_features)].copy()
all_data_clean.shape

(526137, 19)

In [9]:
all_data_clean.isnull().sum()

waktu_setempat          0
id_jalan                0
id_titik_mulai          0
id_titik_akhir          0
Latitude_awal           0
Longitude_awal          0
Latitude_akhir          0
Longitude_akhir         0
highway                 0
lit                     0
maxspeed                0
name                    0
ref                     0
sidewalk           193466
surface            220162
hari_minggu             0
jam                     0
waktu                   0
weekend                 0
dtype: int64

In [10]:
all_data_clean['sidewalk'] = all_data_clean['sidewalk'].fillna('tidak tersedia')
all_data_clean['surface'] = all_data_clean['surface'].fillna('tidak tersedia')

In [11]:
# Check inconsistent data
for col in all_data_clean.select_dtypes(include=['object']):
       print("=="*10)
       print(f" {col}", all_data_clean[col].unique())

 highway ['primary' 'trunk' 'secondary']
 lit ['yes']
 maxspeed ['30 mph' '20 mph']
 name ['High Road' 'Roehampton Lane' 'Upper Richmond Road'
 'Upper Richmond Road West' 'Acre Lane' 'Kings Avenue' 'Tulse Hill'
 'Harrow Road' 'Nightingale Lane' 'Romford Road' 'Albany Street'
 'Clapham Road' 'Finchley Road' 'Brixton Road' 'North End Road'
 'Lavender Hill' 'Brixton Hill' 'Clapham Common South Side'
 'Upper Tooting Road']
 ref ['A1000' 'A306' 'A205' 'A2217' 'B221' 'A204' 'A404' 'B237' 'A118' 'A4201'
 'A3' 'A598' 'A23' 'B317' 'A3036' 'A24']
 sidewalk ['tidak tersedia' 'both' 'separate']
 surface ['tidak tersedia' 'asphalt']


## create jarak menggunakan haversine distance

In [12]:
def haversine_distance(row):
    R = 6371  # Radius bumi dalam kilometer

    lat1 = np.radians(row['Latitude_awal'])
    lon1 = np.radians(row['Longitude_awal'])
    lat2 = np.radians(row['Latitude_akhir'])
    lon2 = np.radians(row['Longitude_akhir'])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

# Menambahkan kolom baru dengan jarak Haversine
all_data_clean['jarak'] = all_data_clean.apply(haversine_distance, axis=1)

In [13]:
all_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526137 entries, 0 to 526136
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   waktu_setempat   526137 non-null  datetime64[ns, UTC]
 1   id_jalan         526137 non-null  int64              
 2   id_titik_mulai   526137 non-null  int64              
 3   id_titik_akhir   526137 non-null  int64              
 4   Latitude_awal    526137 non-null  float64            
 5   Longitude_awal   526137 non-null  float64            
 6   Latitude_akhir   526137 non-null  float64            
 7   Longitude_akhir  526137 non-null  float64            
 8   highway          526137 non-null  object             
 9   lit              526137 non-null  object             
 10  maxspeed         526137 non-null  object             
 11  name             526137 non-null  object             
 12  ref              526137 non-null  object             
 13 

In [14]:
all_data_clean_fix= all_data_clean.drop(columns=['waktu_setempat'])

In [15]:
# Inisialisasi LabelEncoder
from sklearn.preprocessing import LabelEncoder
kolom = all_data_clean_fix.select_dtypes(include=['object']).columns
le = LabelEncoder()
# for column in kolom:
#     all_data_clean_fix[column] = le.fit_transform(all_data_clean_fix[column])
all_data_clean_fix['maxspeed'] = le.fit_transform(all_data_clean_fix['maxspeed'])
all_data_clean_fix = pd.get_dummies(all_data_clean_fix)

In [16]:
all_data_clean_fix.head()

,id_jalan,id_titik_mulai,id_titik_akhir,Latitude_awal,Longitude_awal,Latitude_akhir,Longitude_akhir,maxspeed,hari_minggu,jam,waktu,weekend,jarak,highway_primary,highway_secondary,highway_trunk,lit_yes,name_Acre Lane,name_Albany Street,name_Brixton Hill,name_Brixton Road,name_Clapham Common South Side,name_Clapham Road,name_Finchley Road,name_Harrow Road,name_High Road,name_Kings Avenue,name_Lavender Hill,name_Nightingale Lane,name_North End Road,name_Roehampton Lane,name_Romford Road,name_Tulse Hill,name_Upper Richmond Road,name_Upper Richmond Road West,name_Upper Tooting Road,ref_A1000,ref_A118,ref_A204,ref_A205,ref_A2217,ref_A23,ref_A24,ref_A3,ref_A3036,ref_A306,ref_A404,ref_A4201,ref_A598,ref_B221,ref_B237,ref_B317,sidewalk_both,sidewalk_separate,sidewalk_tidak tersedia,surface_asphalt,surface_tidak tersedia
0,691007296,21390008,1425033102,51.434928,-0.161176,51.434975,-0.161090,1,5,1,4,2,0.007914,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,47010584,1677092762,579493410,51.623990,-0.176398,51.623811,-0.176424,1,5,1,4,2,0.019985,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
2,22932408,26486694,1930267566,51.452530,-0.152437,51.452749,-0.151896,1,5,1,4,2,0.044685,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,142479648,1111592522,3775231113,51.465851,-0.154188,51.465654,-0.155334,1,5,1,4,2,0.082340,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,8504977,5940503398,5940503394,51.530289,-0.228343,51.530197,-0.227949,0,5,1,4,2,0.029118,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


# split train test

In [17]:
train_final = all_data_clean_fix[:ntrain]
test_final = all_data_clean_fix[ntrain:]
print(train_final.shape)
print(test_final.shape)

(398648, 57)
(127489, 57)


In [18]:
X, y = train_final, y_train_final
print(X.shape)
print(y.shape)

(398648, 57)
(398648,)


# Modelling

In [19]:
selected= ['id_jalan', 'id_titik_mulai', 'id_titik_akhir', 'hari_minggu', 'jam',
       'waktu', 'weekend']

In [20]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor()
xgb = XGBRegressor()
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
def smape(actual, predicted):
    absolute_errors = np.abs(predicted - actual)
    scaled_errors = 2 * absolute_errors / (np.abs(predicted) + np.abs(actual))
    return np.mean(scaled_errors) * 100


# Buat scorer khusus untuk sMAPE
smape_scorer = make_scorer(smape, greater_is_better=False)
kf= KFold(n_splits=5, shuffle=True, random_state=42)

## Random Forest Regressor

In [55]:
rfc = RandomForestRegressor(random_state=42)
rfc.fit(X[selected],y)

RandomForestRegressor(random_state=42)

In [56]:
# Latih model pada data training
from sklearn.model_selection import train_test_split

    # Select only the top k features
X_train_selected = X[selected]

# Bagi dataset menjadi data training (75%) dan data test (25%)
X_train, X_test, y_train, y_test = train_test_split(X_train_selected, y, test_size=0.25, random_state=42)
rfc.fit(X_train, y_train)

# Lakukan prediksi pada data test
y_pred_test = rfc.predict(X_test)
y_pred_train = rfc.predict(X_train)

# Hitung SMAPE pada data test dan data training
smape_score_test = smape(y_test, y_pred_test)
smape_score_train = smape(y_train, y_pred_train)

print("SMAPE Score Test:", smape_score_test)
print("SMAPE Score Train:", smape_score_train)

SMAPE Score Test: 9.428214910699452
SMAPE Score Train: 6.4271532541627066


In [60]:
# Perform cross-validation with SMAPE scoring
from sklearn.model_selection import cross_val_score
cross_val_scores = cross_val_score(RandomForestRegressor(random_state=42), X[selected], y, cv=kf, scoring=smape_scorer)

# Print the cross-validation scores
print("Cross-validation scores (SMAPE):", cross_val_scores)
print("Mean SMAPE:", np.mean(cross_val_scores))

Cross-validation scores (SMAPE): [-9.42017469 -9.44619704 -9.40028728 -9.45172165 -9.42812929]
Mean SMAPE: -9.429301991039075


# Tuning Random Forest with Bayesian Optimization

In [ ]:
from time import time
import pprint
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
# A wrapper for measuring time and performances of different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    best_score = optimizer.best_score_
    best_score_std = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start,
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params
# Setting the search space
search_spaces = {
    'n_estimators': Integer(10, 200),
    'max_depth': Integer(1, 20),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 10),
    'max_features': Real(0.1, 1.0),
    'min_impurity_decrease': Real(0.0, 1.0)
}

# Create a RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=42)

# Create BayesSearchCV object
opt = BayesSearchCV(
    rf_regressor,
    search_spaces,
    scoring=smape_scorer,
    cv=kf,
    n_jobs=1,
    n_iter=20,
    return_train_score=False,
    refit=True,
    verbose=100
)

# Report the best parameters
best_params = report_perf(opt, X[selected], y, 'Random Forest Regressor')

In [21]:
rfc_tuning = RandomForestRegressor(random_state=42,max_depth=20,max_features=1.0,
                           min_impurity_decrease=0.0,min_samples_leaf=2,min_samples_split=10,
                           n_estimators=200)
rfc_tuning.fit(X[selected],y)

RandomForestRegressor(max_depth=20, min_samples_leaf=2, min_samples_split=10,
                      n_estimators=200, random_state=42)

In [71]:
# Latih model pada data training
from sklearn.model_selection import train_test_split

    # Select only the top k features
X_train_selected = X[selected]

# Bagi dataset menjadi data training (75%) dan data test (25%)
X_train, X_test, y_train, y_test = train_test_split(X_train_selected, y, test_size=0.25, random_state=42)
rfc_tuning.fit(X_train, y_train)

# Lakukan prediksi pada data test
y_pred_test = rfc_tuning.predict(X_test)
y_pred_train = rfc_tuning.predict(X_train)

# Hitung SMAPE pada data test dan data training
smape_score_test = smape(y_test, y_pred_test)
smape_score_train = smape(y_train, y_pred_train)

print("SMAPE Score Test:", smape_score_test)
print("SMAPE Score Train:", smape_score_train)

SMAPE Score Test: 8.626365082825192
SMAPE Score Train: 7.613435775446837


In [62]:
# Perform cross-validation with SMAPE scoring
from sklearn.model_selection import cross_val_score
cross_val_scores = cross_val_score(rfc_tuning, X[selected], y, cv=kf, scoring=smape_scorer)

# Print the cross-validation scores
print("Cross-validation scores (SMAPE):", cross_val_scores)
print("Mean SMAPE:", np.mean(cross_val_scores))

Cross-validation scores (SMAPE): [-8.61261843 -8.63253387 -8.60575915 -8.62384234 -8.58776843]
Mean SMAPE: -8.612504443909971


# XGBoost

In [63]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X[selected],y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [66]:
# Latih model pada data training
from sklearn.model_selection import train_test_split

    # Select only the top k features
X_train_selected = X[selected]

# Bagi dataset menjadi data training (75%) dan data test (25%)
X_train, X_test, y_train, y_test = train_test_split(X_train_selected, y, test_size=0.25, random_state=42)
xgb.fit(X_train, y_train)

# Lakukan prediksi pada data test
y_pred_test = xgb.predict(X_test)
y_pred_train = xgb.predict(X_train)

# Hitung SMAPE pada data test dan data training
smape_score_test = smape(y_test, y_pred_test)
smape_score_train = smape(y_train, y_pred_train)

print("SMAPE Score Test:", smape_score_test)
print("SMAPE Score Train:", smape_score_train)

SMAPE Score Test: 9.149538646812768
SMAPE Score Train: 9.036033037378882


In [67]:
# Perform cross-validation with SMAPE scoring
from sklearn.model_selection import cross_val_score
cross_val_scores = cross_val_score(xgb, X[selected], y, cv=kf, scoring=smape_scorer)

# Print the cross-validation scores
print("Cross-validation scores (SMAPE):", cross_val_scores)
print("Mean SMAPE:", np.mean(cross_val_scores))

Cross-validation scores (SMAPE): [-9.08412903 -9.11997934 -9.15560158 -9.04918521 -9.07894704]
Mean SMAPE: -9.097568441149729


# XGBoost Tuning with Bayesian Optimization

In [128]:
from time import time
import pprint
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
# A wrapper for measuring time and performances of different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    best_score = optimizer.best_score_
    best_score_std = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start,
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params
# Setting the search space
search_spaces = {
    'n_estimators': Integer(10, 200),
    'learning_rate': Real(0.01, 1.0),
    'max_depth': Integer(1, 20),
    'min_child_weight': Real(0.1, 10),
    'subsample': Real(0.1, 1.0),
    'colsample_bytree': Real(0.1, 1.0)
}

# Create a RandomForestRegressor
xgb = XGBRegressor(random_state=42)

# Create BayesSearchCV object
opt = BayesSearchCV(
    xgb,
    search_spaces,
    scoring=smape_scorer,
    cv=kf,
    n_jobs=1,
    n_iter=20,
    return_train_score=False,
    refit=True,
    verbose=100
)

# Report the best parameters
best_params = report_perf(opt, X[selected], y, 'XGboost')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START colsample_bytree=0.909172884610919, learning_rate=0.893900550443797, max_depth=14, min_child_weight=3.435828492130551, n_estimators=52, subsample=0.46476449803790054
[CV 1/5; 1/1] END colsample_bytree=0.909172884610919, learning_rate=0.893900550443797, max_depth=14, min_child_weight=3.435828492130551, n_estimators=52, subsample=0.46476449803790054;, score=-10.184 total time=  14.7s
[CV 2/5; 1/1] START colsample_bytree=0.909172884610919, learning_rate=0.893900550443797, max_depth=14, min_child_weight=3.435828492130551, n_estimators=52, subsample=0.46476449803790054
[CV 2/5; 1/1] END colsample_bytree=0.909172884610919, learning_rate=0.893900550443797, max_depth=14, min_child_weight=3.435828492130551, n_estimators=52, subsample=0.46476449803790054;, score=-10.291 total time=  15.5s
[CV 3/5; 1/1] START colsample_bytree=0.909172884610919, learning_rate=0.893900550443797, max_depth=14, min_child_weight=3.435828492

In [27]:
xgb_tuning = XGBRegressor(random_state=42,colsample_bytree=0.6094043421124198,learning_rate= 0.4041252174051859,
                         max_depth=11,min_child_weight=7.798752694870117,n_estimators=87,subsample=0.67629599233451)
xgb_tuning.fit(X[selected],y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6094043421124198, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.4041252174051859,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=11, max_leaves=None,
             min_child_weight=7.798752694870117, missing=nan,
             monotone_constraints=None, n_estimators=87, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=42, ...)

In [69]:
# Latih model pada data training
from sklearn.model_selection import train_test_split

    # Select only the top k features
X_train_selected = X[selected]

# Bagi dataset menjadi data training (75%) dan data test (25%)
X_train, X_test, y_train, y_test = train_test_split(X_train_selected, y, test_size=0.25, random_state=42)
xgb_tuning.fit(X_train, y_train)

# Lakukan prediksi pada data test
y_pred_test = xgb_tuning.predict(X_test)
y_pred_train = xgb_tuning.predict(X_train)

# Hitung SMAPE pada data test dan data training
smape_score_test = smape(y_test, y_pred_test)
smape_score_train = smape(y_train, y_pred_train)

print("SMAPE Score Test:", smape_score_test)
print("SMAPE Score Train:", smape_score_train)

SMAPE Score Test: 8.49824332943177
SMAPE Score Train: 7.845625981908055


In [70]:
# Perform cross-validation with SMAPE scoring
from sklearn.model_selection import cross_val_score
cross_val_scores = cross_val_score(xgb_tuning, X[selected], y, cv=kf, scoring=smape_scorer)

# Print the cross-validation scores
print("Cross-validation scores (SMAPE):", cross_val_scores)
print("Mean SMAPE:", np.mean(cross_val_scores))

Cross-validation scores (SMAPE): [-8.44567355 -8.47207052 -8.4608873  -8.4447702  -8.40250954]
Mean SMAPE: -8.44518222199317


In [26]:
test_data = pd.read_csv('/kaggle/input/test-datathon/submission_ensemble2.csv')
print(smape(test_data['rerata_kecepatan'], rfc_tuning_pred))

3.244981929446803


In [29]:
submission = pd.read_csv('/kaggle/input/ristek-datathon-2023/sample_submission.csv')
submission['rerata_kecepatan'] = xgb_tuning_pred
submission.to_csv('submission_new5.csv',index=False)
submission

,id,rerata_kecepatan
0,0,42.131950
1,1,38.774933
2,2,37.035236
3,3,40.798538
4,4,29.648056
...,...,...
127484,127484,31.060932
127485,127485,36.685993
127486,127486,41.272182
127487,127487,33.417778


In [36]:
test_data

,id,rerata_kecepatan
0,0,42.715595
1,1,38.787099
2,2,36.568539
3,3,43.411890
4,4,29.066713
...,...,...
127484,127484,32.355070
127485,127485,36.520379
127486,127486,41.698937
127487,127487,34.480135
